In [4]:
!python3 --version


Python 3.10.12


In [1]:
import sys
print(sys.version)


3.10.12 (main, Feb  4 2025, 14:57:36) [GCC 11.4.0]


In [2]:
import pandas as pd

subset = pd.read_csv("/home/pedige/.local/bin/DQ/DQ/NIFTY/DQ-1/Topic0_train_data.csv")
subset = subset[:700]
subset['date'] = pd.to_datetime(subset['date']).dt.date
subset

,Unnamed: 0,id,date,news,label,pct_change,text_length,textlemma,textlengthforlemma,word_count,topic,topic_name,Entities,IsEntity
0,104,nifty_2,2010-01-11,Research and Markets: Epilepsy - Drug Pipeline...,Fall,-0.0093,91,research and market epilepsy drug pipeline ana...,78,11,0,pharmacy,"['research', 'and', 'market', 'e', '##pile', '...",1
1,127,nifty_2,2010-01-11,Weird Science? Big Pharma's Top Three M&A; D...,Fall,-0.0093,108,weird science big pharma top three deal nears ...,80,13,0,pharmacy,"['weird', 'science', 'big', 'p', '##har', '##m...",1
2,186,nifty_2,2010-01-11,Research and Markets: Booming Hospital Service...,Fall,-0.0093,63,research and market booming hospital service m...,57,8,0,pharmacy,"['research', 'and', 'market', 'boom', '##ing',...",1
3,222,nifty_2,2010-01-11,BioCentury Publishes Investor Picks for New B...,Fall,-0.0093,72,biocentury publishes investor pick for new bio...,61,8,0,pharmacy,"['bio', '##cent', '##ury', 'publishes', 'inves...",1
4,396,nifty_2,2010-01-11,Research and Markets: Vertex Pharmaceuticals -...,Fall,-0.0093,70,research and market vertex pharmaceutical swot...,65,8,0,pharmacy,"['research', 'and', 'market', 'vertex', 'pharm...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,63122,nifty_207,2011-01-20,Money meets innovation at Biotech Showcase 2011,Neutral,0.0023,47,money meet innovation biotech showcase,38,5,0,pharmacy,"['money', 'meet', 'innovation', 'bio', '##tech...",1
696,63168,nifty_207,2011-01-20,Radient Pharmaceuticals Receives Updated Euro...,Neutral,0.0023,119,radient pharmaceutical receives updated europe...,109,15,0,pharmacy,"['r', '##adi', '##ent', 'pharmaceutical', 'rec...",1
697,63221,nifty_207,2011-01-20,"Griffin Securities, Inc. Announces Investment ...",Neutral,0.0023,80,griffin security inc announces investment opin...,71,9,0,pharmacy,"['g', '##riff', '##in', 'security', 'in', '##c...",1
698,63315,nifty_207,2011-01-20,Research and Markets: Bronchitis Therapeutics ...,Neutral,0.0023,96,research and market bronchitis therapeutic pip...,82,10,0,pharmacy,"['research', 'and', 'market', 'br', '##on', '#...",1


In [3]:
import pandas as pd
import torch
from itertools import combinations
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

filtered_entities = subset[subset['IsEntity'] == 1][['Entities', 'date', 'textlemma']] #Here the filtration for 1s from the NER)

def calculate_sbert_similarity(entities1, entities2):
    if not entities1 or not entities2:
        return 0  # Return 0 similarity if either entity list is empty

    text1 = " ".join(entities1)
    text2 = " ".join(entities2)

    embedding1 = model.encode(text1, convert_to_tensor=True)
    embedding2 = model.encode(text2, convert_to_tensor=True)

    similarity=util.pytorch_cos_sim(embedding1, embedding2).item()
    return similarity

# Compare entity similarities between articles
similarities = []
pairs = list(combinations(filtered_entities.index, 2))

for i, j in pairs:
    entities1 = filtered_entities.loc[i, "Entities"]
    entities2 = filtered_entities.loc[j, "Entities"]

    if len(entities1) > 1 and len(entities2) > 1:
        sim = calculate_sbert_similarity(entities1, entities2)
        date_difference = abs((filtered_entities.loc[i, "date"] - filtered_entities.loc[j, "date"]).days)

        similarities.append({
            "Entity1": entities1,
            "Entity2": entities2,
            "Article_1_News": filtered_entities.loc[i, "textlemma"],
            "Article_2_News": filtered_entities.loc[j, "textlemma"],
            "Date_Difference": date_difference,
            "SBERT_Similarity": sim})

similarity_df = pd.DataFrame(similarities)
high_similarity = similarity_df.query('SBERT_Similarity > 0.99')
print(high_similarity.head())

2025-04-01 18:06:45.504895: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-01 18:06:45.518396: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743530805.535012  205883 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743530805.540234  205883 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743530805.552637  205883 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

                                                 Entity1  \
201    ['research', 'and', 'market', 'e', '##pile', '...   
204    ['research', 'and', 'market', 'e', '##pile', '...   
7647   ['f', '##da', 'un', '##ve', '##ils', 'first', ...   
10382  ['ah', '##f', 'blast', 'me', '##rc', '##k', 'a...   
16489  ['bio', '##gen', 'id', '##ec', 'provide', 'bus...   

                                                 Entity2  \
201    ['research', 'and', 'market', 'ne', '##ut', '#...   
204    ['research', 'and', 'market', 'pan', '##cre', ...   
7647   ['f', '##da', 'un', '##ve', '##ils', 'first', ...   
10382  ['ah', '##f', 'blast', 'me', '##rc', '##k', 'a...   
16489  ['bio', '##gen', 'id', '##ec', 'provides', 'bu...   

                                          Article_1_News  \
201    research and market epilepsy drug pipeline ana...   
204    research and market epilepsy drug pipeline ana...   
7647     fda unveils first phase transparency initiative   
10382  ahf blast merck aid drug pricin

In [4]:
# Save the DataFrame to a pickle file
similarity_df.to_pickle('similarity_results.pkl')

In [5]:
print(high_similarity.loc[high_similarity.index == 7647, ["Article_1_News", "Article_2_News"]])

                                       Article_1_News  \
7647  fda unveils first phase transparency initiative   

                                       Article_2_News  
7647  fda unveils first phase transparency initiative  


In [6]:
import pandas as pd
import torch
from itertools import combinations
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MPNet-base-v2')

filtered_entities = subset[subset['IsEntity'] == 1][['Entities', 'date', 'textlemma']] #Here the filtration for 1s from the NER)

def calculate_sbert_similarity(entities1, entities2):
    if not entities1 or not entities2:
        return 0  # Return 0 similarity if either entity list is empty

    text1 = " ".join(entities1)
    text2 = " ".join(entities2)

    embedding1 = model.encode(text1, convert_to_tensor=True)
    embedding2 = model.encode(text2, convert_to_tensor=True)

    similarity=util.pytorch_cos_sim(embedding1, embedding2).item()
    return similarity

# Compare entity similarities between articles
similarities = []
pairs = list(combinations(filtered_entities.index, 2))

for i, j in pairs:
    entities1 = filtered_entities.loc[i, "Entities"]
    entities2 = filtered_entities.loc[j, "Entities"]

    if len(entities1) > 1 and len(entities2) > 1:
        sim = calculate_sbert_similarity(entities1, entities2)
        date_difference = abs((filtered_entities.loc[i, "date"] - filtered_entities.loc[j, "date"]).days)

        similarities.append({
            "Entity1": entities1,
            "Entity2": entities2,
            "Article_1_News": filtered_entities.loc[i, "textlemma"],
            "Article_2_News": filtered_entities.loc[j, "textlemma"],
            "Date_Difference": date_difference,
            "SBERT_Similarity": sim})

similarity_df = pd.DataFrame(similarities)
high_similarity = similarity_df.query('SBERT_Similarity > 0.99')
print(high_similarity.head())

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

                                                 Entity1  \
201    ['research', 'and', 'market', 'e', '##pile', '...   
7647   ['f', '##da', 'un', '##ve', '##ils', 'first', ...   
10382  ['ah', '##f', 'blast', 'me', '##rc', '##k', 'a...   
16489  ['bio', '##gen', 'id', '##ec', 'provide', 'bus...   
25190  ['research', 'and', 'market', 'brain', 'tumor'...   

                                                 Entity2  \
201    ['research', 'and', 'market', 'ne', '##ut', '#...   
7647   ['f', '##da', 'un', '##ve', '##ils', 'first', ...   
10382  ['ah', '##f', 'blast', 'me', '##rc', '##k', 'a...   
16489  ['bio', '##gen', 'id', '##ec', 'provides', 'bu...   
25190  ['research', 'and', 'market', 'brain', 'tumor'...   

                                          Article_1_News  \
201    research and market epilepsy drug pipeline ana...   
7647     fda unveils first phase transparency initiative   
10382  ahf blast merck aid drug pricing during invest...   
16489  biogen idec provide business up

In [7]:
print(high_similarity.loc[high_similarity.index == 7647, ["Article_1_News", "Article_2_News"]])

                                       Article_1_News  \
7647  fda unveils first phase transparency initiative   

                                       Article_2_News  
7647  fda unveils first phase transparency initiative  


In [8]:
# Save the DataFrame to a pickle file
similarity_df.to_pickle('similarity_results_with_all_mpnet_base_v2.pkl')